In [17]:
# Read environment variables

import os
import traceback

import openai
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

openai_url      = os.environ['AZURE_OPENAI_URL']
openai_key      = os.environ['AZURE_OPENAI_KEY']
openai_version  = os.environ['AZURE_OPENAI_API_VERSION']  # "2023-07-01-preview" 
embeddings_dep  = os.environ['AZURE_OPENAI_EMBEDDINGS_DEP']
completions_dep = os.environ['AZURE_OPENAI_COMPLETIONS_DEP']

openai.api_type = "azure"
openai.api_version = openai_version
openai.api_base = openai_url
openai.api_key  = openai_key

print("openai_url:      {}".format(openai_url))
print("openai_key:      {}".format(openai_key[0:3]))
print("openai_version:  {}".format(openai_version))
print("embeddings_dep:  {}".format(embeddings_dep))
print("completions_dep: {}".format(completions_dep))

try:
    aoai_client = AzureOpenAI(
        azure_endpoint=openai_url,
        api_key=openai_key,
        api_version=openai_version,
    )
    print("aoai_client created")
except Exception as e:
    print("Exception while creating AzureOpenAI client: {}".format(str(e)))
    print(traceback.format_exc())


openai_url:      https://gbbcjcaigopenai3.openai.azure.com/
openai_key:      d1b
openai_version:  2023-07-01-preview
embeddings_dep:  embeddings
completions_dep: gpt4
aoai_client created


In [18]:
# Generate embeddings for a given string value.

import traceback
from pysrc.util.fs import FS

try:
    text = "A long time ago came a man on a track. Walking thirty miles with a sack on his back."
    response = aoai_client.embeddings.create(input=text, model=embeddings_dep)
    embedding = response.data[0].embedding

    print("embedding length: {}".format(len(embedding)))   
    print("embedding[:4]: {}".format(embedding[:4])) 
    FS.write_json(embedding, "tmp/embedding.json")
except Exception as e:
    print("Exception while creating embedding: {}".format(str(e)))
    print(traceback.format_exc())


embedding length: 1536
embedding[:4]: [0.018863078206777573, -0.006897789891809225, 0.015606330707669258, 0.007549139205366373]


In [19]:
# Generate a simple completion

import json
import traceback

try:
    completion = aoai_client.chat.completions.create(
        model=completions_dep,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "What is the capitol of North Carolina?"},
        ],
        temperature=0
    )
    FS.write_json(json.loads(completion.to_json()), "tmp/simple_completion.json")
except Exception as e:
    print("Exception while creating completion: {}".format(str(e)))
    print(traceback.format_exc())

In [20]:
# Generate a python module 

import json
import traceback

try:
    prompt = """
You have two tasks to execute:
- Generate a python 3 class named FSUtil in module pysrc.util.fsutil.py.
- Also generate a unit test for this class which uses the pytest testing framework.

The focus of class FSUtil is on providing easy to use filesystem access to read and write files.

The class has no instance methods, only class methods.

All methods should assume the following:
- Use "utf-8" encoding

It should implement the following list of methods per these signatures:
- def as_unix_filename(cls, filename: str) -> str:
- def read(cls, infile: str) -> str | None:
- def read_with_encoding, infile: str, encoding: str) -> str | None:
- def read_lines(cls, infile: str) -> list[str] | None:
"""
    # AzureOpenAI chat.completions.create 
    completion = aoai_client.chat.completions.create(
        model=completions_dep,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who generates Python code."},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    FS.write("tmp/fsutil.py", completion.choices[0].message.content)
    FS.write_json(json.loads(completion.to_json()), "tmp/fsutil_completion.json")
except Exception as e:
    print("Exception while creating completion: {}".format(str(e)))
    print(traceback.format_exc())